# ExLlamaV2

[ExLlamav2](https://github.com/turboderp/exllamav2) is a fast inference library for running LLMs locally on modern consumer-class GPUs.

It supports inference for GPTQ & EXL2 quantized models, which can be accessed on [Hugging Face](https://huggingface.co/TheBloke).

This notebook goes over how to run `exllamav2` within LangChain.

Additional information:
[ExLlamav2 examples](https://github.com/turboderp/exllamav2/tree/master/examples)


## Installation

Refer to the official [doc](https://github.com/turboderp/exllamav2)
For this notebook, the requirements are :
- python 3.11
- langchain 0.1.7
- CUDA: 12.1.0 (see bellow)
- torch==2.1.1+cu121
- exllamav2 (0.0.12+cu121)

If you want to install the same exllamav2 version :
```shell
pip install https://github.com/turboderp/exllamav2/releases/download/v0.0.12/exllamav2-0.0.12+cu121-cp311-cp311-linux_x86_64.whl
```

if you use conda, the dependencies are :
```
  - conda-forge::ninja
  - nvidia/label/cuda-12.1.0::cuda
  - conda-forge::ffmpeg
  - conda-forge::gxx=11.4
```

## Usage

You don't need an `API_TOKEN` as you will run the LLM locally.

It is worth understanding which models are suitable to be used on the desired machine.

[TheBloke's](https://huggingface.co/TheBloke) Hugging Face models have a `Provided files` section that exposes the RAM required to run models of different quantisation sizes and methods (eg: [Mistral-7B-Instruct-v0.2-GPTQ](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GPTQ)).


In [3]:
!pip install -U langchain huggingface_hub exllamav2 langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/langchain-ai/langchain.git

Cloning into 'langchain'...
remote: Enumerating objects: 234693, done.
remote: Counting objects: 100% (829/829), done.
remote: Compressing objects: 100% (342/342), done.
remote: Total 234693 (delta 613), reused 492 (delta 487), pack-reused 233864 (from 2)
Receiving objects: 100% (234693/234693), 403.70 MiB | 31.50 MiB/s, done.
Resolving deltas: 100% (177404/177404), done.
Updating files: 100% (7411/7411), done.


In [6]:
import os

from huggingface_hub import snapshot_download
from langchain_community.llms.exllamav2 import ExLlamaV2
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
#from libs.langchain.langchain.chains.llm import LLMChain

In [7]:
# function to download the gptq model
def download_GPTQ_model(model_name: str, models_dir: str = "./models/") -> str:
    """Download the model from hugging face repository.

    Params:
    model_name: str: the model name to download (repository name). Example: "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"
    """
    # Split the model name and create a directory name. Example: "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ" -> "TheBloke_CapybaraHermes-2.5-Mistral-7B-GPTQ"

    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    _model_name = model_name.split("/")
    _model_name = "_".join(_model_name)
    model_path = os.path.join(models_dir, _model_name)
    if _model_name not in os.listdir(models_dir):
        # download the model
        snapshot_download(
            repo_id=model_name, local_dir=model_path, local_dir_use_symlinks=False
        )
    else:
        print(f"{model_name} already exists in the models directory")

    return model_path

In [8]:
from exllamav2.generator import (
    ExLlamaV2Sampler,
)

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.85
settings.top_k = 50
settings.top_p = 0.8
settings.token_repetition_penalty = 1.05

model_path = download_GPTQ_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")

callbacks = [StreamingStdOutCallbackHandler()]

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Verbose is required to pass to the callback manager
llm = ExLlamaV2(
    model_path=model_path,
    callbacks=callbacks,
    verbose=True,
    settings=settings,
    streaming=True,
    max_new_tokens=150,
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What Football team won the UEFA Champions League in the year the iphone 6s was released?"

output = llm_chain.invoke({"question": question})
print(output)

Output()

Loading exllamav2_ext extension (JIT)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

{'token_repetition_penalty': 1.05, 'token_repetition_range': -1, 'token_repetition_decay': 0, 'token_frequency_penalty': 0.0, 'token_presence_penalty': 0.0, 'temperature': 0.85, 'smoothing_factor': 0.0, 'min_temp': 0, 'max_temp': 0.0, 'temp_exponent': 1.0, 'top_k': 50, 'top_p': 0.8, 'top_a': 0.0, 'min_p': 0, 'tfs': 0, 'typical': 0, 'skew': 0, 'temperature_last': False, 'mirostat': False, 'mirostat_tau': 1.5, 'mirostat_eta': 0.1, 'mirostat_mu': None, 'token_bias': None, 'cfg_scale': None, 'post_sampling_hooks': [], 'dry_allowed_length': 2, 'dry_base': 1.75, 'dry_multiplier': 0.0, 'dry_sequence_breakers': None, 'dry_range': 0, 'dry_max_ngram': 20, 'ngram_trie': None, 'ngram_index': 0, 'ngram_history': deque([]), 'xtc_probability': 0.0, 'xtc_threshold': 0.1, 'xtc_ignore_tokens': None}
stop_sequences []


<ipython-input-8-bad80366ee82>:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


 We know that the iPhone 6s was released on September 25, 2015. The UEFA Champions League final match is usually held in May of each year.

Let's see which team won the UEFA Champions League in May 2015 or earlier:

1. Barcelona (2014-15)
2. Real Madrid (2013-14)
3. Bayern Munich (2012-13)
4. Chelsea (2011-12)
5. Inter Milan (2009-10)

None of these teams won the UEFA Champions League in May 2015 or{'question': 'What Football team won the UEFA Champions League in the year the iphone 6s was released?', 'text': " We know that the iPhone 6s was released on September 25, 2015. The UEFA Champions League final match is usually held in May of each year.\n\nLet's see which team won the UEFA Champions League in May 2015 or earlier:\n\n1. Barcelona (2014-15)\n2. Real Madrid (2013-14)\n3. Bayern Munich (2012-13)\n4. Chelsea (2011-12)\n5. Inter Milan (2009-10)\n\nNone of these teams won the UEFA Champions League in May 2015 or"}


In [9]:
import gc

import torch

torch.cuda.empty_cache()
gc.collect()
!nvidia-smi

Thu Apr 10 03:35:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             26W /   70W |    8278MiB /  15360MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from exllamav2.generator import (
    ExLlamaV2Sampler,
)

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.85
settings.top_k = 50
settings.top_p = 0.8
settings.token_repetition_penalty = 1.05

model_path = download_GPTQ_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")

callbacks = [StreamingStdOutCallbackHandler()]

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Verbose is required to pass to the callback manager
llm = ExLlamaV2(
    model_path=model_path,
    callbacks=callbacks,
    verbose=True,
    settings=settings,
    streaming=True,
    max_new_tokens=150,
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What Football team won the UEFA Champions League in the year the iphone 6s was released?"

output = llm_chain.invoke({"question": question})
print(output)

In [10]:
from exllamav2.generator import ExLlamaV2Sampler
from langchain.llms import ExLlamaV2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
import os

# Helper function to download model (you can replace this with your own implementation)
def download_GPTQ_model(model_name):
    # Implement model download or just return the local path if already downloaded
    # For example:
    base_path = "models/"
    model_path = os.path.join(base_path, model_name.split("/")[-1])
    if not os.path.exists(model_path):
        print(f"Please download the model from {model_name} and place it in {model_path}")
    return model_path

# 1. Load and process the PDF file
def load_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    chunks = text_splitter.split_documents(documents)

    return chunks

# 2. Create embeddings and vector store
def create_vector_store(pdf_chunks):
    # Load embeddings model (you can use any model compatible with your hardware)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda'} # Use 'cpu' if you don't have GPU
    )

    # Create vector store
    vector_store = FAISS.from_documents(pdf_chunks, embeddings)

    return vector_store

# 3. Configure ExLlamaV2 model
def setup_llm():
    settings = ExLlamaV2Sampler.Settings()
    settings.temperature = 0.85
    settings.top_k = 50
    settings.top_p = 0.8
    settings.token_repetition_penalty = 1.05

    model_path = download_GPTQ_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")
    callbacks = [StreamingStdOutCallbackHandler()]

    # Create LLM
    llm = ExLlamaV2(
        model_path=model_path,
        callbacks=callbacks,
        verbose=True,
        settings=settings,
        streaming=True,
        max_new_tokens=500,  # Increased for more detailed responses
    )

    return llm

# 4. RAG Chain
def run_rag_chain(pdf_path, question):
    # Process PDF
    chunks = load_pdf(pdf_path)
    vector_store = create_vector_store(chunks)

    # Retrieve relevant documents
    relevant_docs = vector_store.similarity_search(question, k=3)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # Setup LLM
    llm = setup_llm()

    # Create prompt template for RAG
    template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context:
    {context}

    Question: {question}

    Answer: Let's think step by step."""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    # Create and run chain
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    output = llm_chain.invoke({"context": context, "question": question})

    return output

# Example usage
if __name__ == "__main__":
    pdf_path = "your_document.pdf"  # Replace with your PDF file path
    question = "What are the key points in this document?"  # Replace with your question

    result = run_rag_chain(pdf_path, question)
    print("\n\nFinal answer:")
    print(result['text'])

ImportError: cannot import name 'ExLlamaV2' from 'langchain.llms' (/usr/local/lib/python3.11/dist-packages/langchain/llms/__init__.py)

In [14]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 8.0 MB/s eta 0:00:00


In [16]:
!pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [17]:
!pip install chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.5 MB/s eta 0:00:00


In [19]:
!git clone https://github.com/facebookresearch/faiss.git
%cd faiss && cmake -B build . -DFAISS_ENABLE_GPU=ON -DFAISS_ENABLE_PYTHON=ON -DCMAKE_BUILD_TYPE=Release && cmake --build build --config Release -j 8 && cd build/faiss/python && pip install .
!cmake --build build --config Release -j 8
%cd build/faiss/python
!pip install .

Cloning into 'faiss'...
remote: Enumerating objects: 65895, done.
remote: Counting objects: 100% (32575/32575), done.
remote: Compressing objects: 100% (815/815), done.
remote: Total 65895 (delta 32242), reused 31767 (delta 31760), pack-reused 33320 (from 3)
Receiving objects: 100% (65895/65895), 231.63 MiB | 30.06 MiB/s, done.
Resolving deltas: 100% (60167/60167), done.
[Errno 2] No such file or directory: 'faiss && cmake -B build . -DFAISS_ENABLE_GPU=ON -DFAISS_ENABLE_PYTHON=ON -DCMAKE_BUILD_TYPE=Release && cmake --build build --config Release -j 8 && cd build/faiss/python && pip install .'
/content
Error: /content/build is not a directory
[Errno 2] No such file or directory: 'build/faiss/python'
/content
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [11]:
#!pip install langchain_community

from exllamav2.generator import ExLlamaV2Sampler
# Import ExLlamaV2 from langchain_community instead of langchain.llms
from langchain_community.llms.exllamav2 import ExLlamaV2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
import os

# ... (rest of the code remains the same) ...

In [15]:
# Helper function to download model (you can replace this with your own implementation)
def download_GPTQ_model(model_name):
    # Implement model download or just return the local path if already downloaded
    # For example:
    base_path = "models/"
    model_path = os.path.join(base_path, model_name.split("/")[-1])
    if not os.path.exists(model_path):
        print(f"Please download the model from {model_name} and place it in {model_path}")
    return model_path

# 1. Load and process the PDF file
def load_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    chunks = text_splitter.split_documents(documents)

    return chunks

# 2. Create embeddings and vector store
def create_vector_store(pdf_chunks):
    # Load embeddings model (you can use any model compatible with your hardware)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda'} # Use 'cpu' if you don't have GPU
    )

    # Create vector store
    vector_store = FAISS.from_documents(pdf_chunks, embeddings)

    return vector_store

# 3. Configure ExLlamaV2 model
def setup_llm():
    settings = ExLlamaV2Sampler.Settings()
    settings.temperature = 0.85
    settings.top_k = 50
    settings.top_p = 0.8
    settings.token_repetition_penalty = 1.05

    model_path = download_GPTQ_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")
    callbacks = [StreamingStdOutCallbackHandler()]

    # Create LLM
    llm = ExLlamaV2(
        model_path=model_path,
        callbacks=callbacks,
        verbose=True,
        settings=settings,
        streaming=True,
        max_new_tokens=500,  # Increased for more detailed responses
    )

    return llm

# 4. RAG Chain
def run_rag_chain(pdf_path, question):
    # Process PDF
    chunks = load_pdf(pdf_path)
    vector_store = create_vector_store(chunks)

    # Retrieve relevant documents
    relevant_docs = vector_store.similarity_search(question, k=3)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # Setup LLM
    llm = setup_llm()

    # Create prompt template for RAG
    template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context:
    {context}

    Question: {question}

    Answer: Let's think step by step."""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    # Create and run chain
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    output = llm_chain.invoke({"context": context, "question": question})

    return output

# Example usage
if __name__ == "__main__":
    pdf_path = "/content/The_Little_Prince_Antoine_de_Saint_Exupery.pdf"  # Replace with your PDF file path
    question = "What are the key points in this document?"  # Replace with your question

    result = run_rag_chain(pdf_path, question)
    print("\n\nFinal answer:")
    print(result['text'])

<ipython-input-15-09e872a3864a>:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
from exllamav2.generator import ExLlamaV2Sampler
from langchain.llms import ExLlamaV2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma  # Replacing FAISS with Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
import os

In [ ]:
from exllamav2.generator import ExLlamaV2Sampler
# Import ExLlamaV2 from langchain_community instead of langchain.llms
from langchain_community.llms.exllamav2 import ExLlamaV2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
import os

In [23]:
from exllamav2.generator import ExLlamaV2Sampler
# Import ExLlamaV2 from langchain_community instead of langchain.llms
from langchain_community.llms.exllamav2 import ExLlamaV2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
import os

# Import Chroma here
from langchain.vectorstores import Chroma

# ... (rest of the code remains the same) ...

In [26]:
from exllamav2.generator import (
    ExLlamaV2Sampler,
)

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.85
settings.top_k = 50
settings.top_p = 0.8
settings.token_repetition_penalty = 1.05

model_path = download_GPTQ_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")

Please download the model from TheBloke/Mistral-7B-Instruct-v0.2-GPTQ and place it in /content/models/Mistral-7B-Instruct-v0.2-GPTQ


In [27]:




# Helper function to download model (you can replace this with your own implementation)
def download_GPTQ_model(model_name):
    # Implement model download or just return the local path if already downloaded
    # For example:
    base_path = "/content/models"
    model_path = os.path.join(base_path, model_name.split("/")[-1])
    if not os.path.exists(model_path):
        print(f"Please download the model from {model_name} and place it in {model_path}")
    return model_path

# 1. Load and process the PDF file
def load_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    chunks = text_splitter.split_documents(documents)

    return chunks

# 2. Create embeddings and vector store
def create_vector_store(pdf_chunks):
    # Load embeddings model (you can use any model compatible with your hardware)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda'} # Use 'cpu' if you don't have GPU
    )

    # Create vector store using Chroma instead of FAISS
    vector_store = Chroma.from_documents(pdf_chunks, embeddings)

    return vector_store

# 3. Configure ExLlamaV2 model
def setup_llm():
    settings = ExLlamaV2Sampler.Settings()
    settings.temperature = 0.85
    settings.top_k = 50
    settings.top_p = 0.8
    settings.token_repetition_penalty = 1.05

    model_path = download_GPTQ_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")
    callbacks = [StreamingStdOutCallbackHandler()]

    # Create LLM
    llm = ExLlamaV2(
        model_path=model_path,
        callbacks=callbacks,
        verbose=True,
        settings=settings,
        streaming=True,
        max_new_tokens=500,  # Increased for more detailed responses
    )

    return llm

# 4. RAG Chain
def run_rag_chain(pdf_path, question):
    # Process PDF
    chunks = load_pdf(pdf_path)
    vector_store = create_vector_store(chunks)

    # Retrieve relevant documents
    relevant_docs = vector_store.similarity_search(question, k=3)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # Setup LLM
    llm = setup_llm()

    # Create prompt template for RAG
    template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context:
    {context}

    Question: {question}

    Answer: Let's think step by step."""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    # Create and run chain
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    output = llm_chain.invoke({"context": context, "question": question})

    return output

# Example usage
if __name__ == "__main__":
    pdf_path = "/content/The_Little_Prince_Antoine_de_Saint_Exupery.pdf"  # Replace with your PDF file path
    question = "What are the key points in this document?"  # Replace with your question

    result = run_rag_chain(pdf_path, question)
    print("\n\nFinal answer:")
    print(result['text'])

Please download the model from TheBloke/Mistral-7B-Instruct-v0.2-GPTQ and place it in /content/models/Mistral-7B-Instruct-v0.2-GPTQ
{'token_repetition_penalty': 1.05, 'token_repetition_range': -1, 'token_repetition_decay': 0, 'token_frequency_penalty': 0.0, 'token_presence_penalty': 0.0, 'temperature': 0.85, 'smoothing_factor': 0.0, 'min_temp': 0, 'max_temp': 0.0, 'temp_exponent': 1.0, 'top_k': 50, 'top_p': 0.8, 'top_a': 0.0, 'min_p': 0, 'tfs': 0, 'typical': 0, 'skew': 0, 'temperature_last': False, 'mirostat': False, 'mirostat_tau': 1.5, 'mirostat_eta': 0.1, 'mirostat_mu': None, 'token_bias': None, 'cfg_scale': None, 'post_sampling_hooks': [], 'dry_allowed_length': 2, 'dry_base': 1.75, 'dry_multiplier': 0.0, 'dry_sequence_breakers': None, 'dry_range': 0, 'dry_max_ngram': 20, 'ngram_trie': None, 'ngram_index': 0, 'ngram_history': deque([]), 'xtc_probability': 0.0, 'xtc_threshold': 0.1, 'xtc_ignore_tokens': None}


ValidationError: 1 validation error for ExLlamaV2
  Assertion failed, Can't find /content/models/Mistral-7B-Instruct-v0.2-GPTQ [type=assertion_error, input_value={'model_path': '/content/...isallowed_tokens': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/assertion_error

In [28]:
from exllamav2.generator import ExLlamaV2Sampler
# Import ExLlamaV2 from langchain_community instead of langchain.llms
from langchain_community.llms.exllamav2 import ExLlamaV2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
import os
import torch
import huggingface_hub
from huggingface_hub import hf_hub_download, snapshot_download

# Helper function to download model properly from Hugging Face
def download_GPTQ_model(model_name):
    """Download a model from Hugging Face and return the local path."""
    base_path = "/content/models"
    model_dir = os.path.join(base_path, model_name.split("/")[-1])

    if not os.path.exists(base_path):
        os.makedirs(base_path, exist_ok=True)

    # Check if model is already downloaded
    if os.path.exists(model_dir) and len(os.listdir(model_dir)) > 0:
        print(f"Model already exists at {model_dir}")
        return model_dir

    try:
        # Download the model files from Hugging Face
        print(f"Downloading model {model_name} to {model_dir}...")
        snapshot_download(
            repo_id=model_name,
            local_dir=model_dir,
            local_dir_use_symlinks=False
        )
        print(f"Model downloaded successfully to {model_dir}")
        return model_dir
    except Exception as e:
        print(f"Error downloading model: {e}")
        print("Please download the model manually from Hugging Face")
        # Create directory anyway so code can continue
        os.makedirs(model_dir, exist_ok=True)
        return model_dir

# 1. Load and process the PDF file
def load_pdf(pdf_path):
    """Load and chunk a PDF file."""
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF file not found: {pdf_path}")

    print(f"Loading PDF from {pdf_path}...")
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from PDF")

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks")

    return chunks

# 2. Create embeddings and vector store
def create_vector_store(pdf_chunks):
    """Create a vector store from document chunks."""
    print("Creating embeddings...")

    # Check if CUDA is available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load embeddings model
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': device}
    )

    # Create vector store
    print("Creating vector store...")
    vector_store = Chroma.from_documents(pdf_chunks, embeddings)
    print("Vector store created successfully")

    return vector_store

# 3. Configure ExLlamaV2 model
def setup_llm(model_name="TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"):
    """Set up the ExLlamaV2 model."""
    print("Setting up LLM...")

    # Configure sampling settings
    settings = ExLlamaV2Sampler.Settings()
    settings.temperature = 0.85
    settings.top_k = 50
    settings.top_p = 0.8
    settings.token_repetition_penalty = 1.05

    # Download model
    model_path = download_GPTQ_model(model_name)
    callbacks = [StreamingStdOutCallbackHandler()]

    try:
        # Create LLM with proper error handling
        llm = ExLlamaV2(
            model_path=model_path,
            callbacks=callbacks,
            verbose=True,
            settings=settings,
            streaming=True,
            max_new_tokens=500,
        )
        print("LLM set up successfully")
        return llm
    except Exception as e:
        print(f"Error setting up LLM: {e}")
        print("\nTroubleshooting steps:")
        print("1. Check if model files are properly downloaded")
        print("2. Verify the model path is correct")
        print("3. Ensure you have the right version of ExLlamaV2")
        print("4. Make sure you have enough GPU memory")
        raise

# 4. RAG Chain
def run_rag_chain(pdf_path, question, model_name="TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"):
    """Run a RAG chain to answer a question based on PDF content."""
    print(f"Processing question: '{question}'")

    # Process PDF
    chunks = load_pdf(pdf_path)
    vector_store = create_vector_store(chunks)

    # Retrieve relevant documents
    print("Retrieving relevant document chunks...")
    relevant_docs = vector_store.similarity_search(question, k=3)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    print(f"Retrieved {len(relevant_docs)} relevant chunks")

    # Setup LLM
    llm = setup_llm(model_name)

    # Create prompt template for RAG
    template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context:
    {context}

    Question: {question}

    Answer: Let's think step by step."""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    # Create and run chain
    print("Running LLM chain...")
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    output = llm_chain.invoke({"context": context, "question": question})

    return output

# Example usage
if __name__ == "__main__":
    pdf_path = "/content/The_Little_Prince_Antoine_de_Saint_Exupery.pdf"  # Replace with your PDF file path
    question = "What are the key points in this document?"  # Replace with your question

    try:
        result = run_rag_chain(pdf_path, question)
        print("\n\nFinal answer:")
        print(result['text'])
    except Exception as e:
        print(f"Error running RAG chain: {e}")

Processing question: 'What are the key points in this document?'
Loading PDF from /content/The_Little_Prince_Antoine_de_Saint_Exupery.pdf...
Loaded 54 pages from PDF
Split into 111 chunks
Creating embeddings...
Using device: cuda
Creating vector store...
Vector store created successfully
Retrieving relevant document chunks...
Retrieved 3 relevant chunks
Setting up LLM...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/23.1k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Model downloaded successfully to /content/models/Mistral-7B-Instruct-v0.2-GPTQ
{'token_repetition_penalty': 1.05, 'token_repetition_range': -1, 'token_repetition_decay': 0, 'token_frequency_penalty': 0.0, 'token_presence_penalty': 0.0, 'temperature': 0.85, 'smoothing_factor': 0.0, 'min_temp': 0, 'max_temp': 0.0, 'temp_exponent': 1.0, 'top_k': 50, 'top_p': 0.8, 'top_a': 0.0, 'min_p': 0, 'tfs': 0, 'typical': 0, 'skew': 0, 'temperature_last': False, 'mirostat': False, 'mirostat_tau': 1.5, 'mirostat_eta': 0.1, 'mirostat_mu': None, 'token_bias': None, 'cfg_scale': None, 'post_sampling_hooks': [], 'dry_allowed_length': 2, 'dry_base': 1.75, 'dry_multiplier': 0.0, 'dry_sequence_breakers': None, 'dry_range': 0, 'dry_max_ngram': 20, 'ngram_trie': None, 'ngram_index': 0, 'ngram_history': deque([]), 'xtc_probability': 0.0, 'xtc_threshold': 0.1, 'xtc_ignore_tokens': None}
Error setting up LLM: Insufficient VRAM for model and cache

Troubleshooting steps:
1. Check if model files are properly downloa

### aشغال جيد

In [1]:
from exllamav2.generator import ExLlamaV2Sampler
# Import ExLlamaV2 from langchain_community instead of langchain.llms
from langchain_community.llms.exllamav2 import ExLlamaV2
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
import os
import torch
import huggingface_hub
from huggingface_hub import hf_hub_download, snapshot_download

# Helper function to download model properly from Hugging Face
def download_GPTQ_model(model_name):
    """Download a model from Hugging Face and return the local path."""
    base_path = "/content/models"
    model_dir = os.path.join(base_path, model_name.split("/")[-1])

    if not os.path.exists(base_path):
        os.makedirs(base_path, exist_ok=True)

    # Check if model is already downloaded
    if os.path.exists(model_dir) and len(os.listdir(model_dir)) > 0:
        print(f"Model already exists at {model_dir}")
        return model_dir

    try:
        # Download the model files from Hugging Face
        print(f"Downloading model {model_name} to {model_dir}...")
        snapshot_download(
            repo_id=model_name,
            local_dir=model_dir,
            local_dir_use_symlinks=False
        )
        print(f"Model downloaded successfully to {model_dir}")
        return model_dir
    except Exception as e:
        print(f"Error downloading model: {e}")
        print("Please download the model manually from Hugging Face")
        # Create directory anyway so code can continue
        os.makedirs(model_dir, exist_ok=True)
        return model_dir

# 1. Load and process the PDF file
def load_pdf(pdf_path):
    """Load and chunk a PDF file."""
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF file not found: {pdf_path}")

    print(f"Loading PDF from {pdf_path}...")
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from PDF")

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks")

    return chunks

# 2. Create embeddings and vector store
def create_vector_store(pdf_chunks):
    """Create a vector store from document chunks."""
    print("Creating embeddings...")

    # Check if CUDA is available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load embeddings model
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': device}
    )

    # Create vector store
    print("Creating vector store...")
    vector_store = Chroma.from_documents(pdf_chunks, embeddings)
    print("Vector store created successfully")

    return vector_store

# 3. Configure ExLlamaV2 model
def setup_llm(model_name="TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"):
    """Set up the ExLlamaV2 model."""
    print("Setting up LLM...")

    # Configure sampling settings
    settings = ExLlamaV2Sampler.Settings()
    settings.temperature = 0.85
    settings.top_k = 50
    settings.top_p = 0.8
    settings.token_repetition_penalty = 1.05

    # Download model
    model_path = download_GPTQ_model(model_name)
    callbacks = [StreamingStdOutCallbackHandler()]

    try:
        # Create LLM with proper error handling
        llm = ExLlamaV2(
            model_path=model_path,
            callbacks=callbacks,
            verbose=True,
            settings=settings,
            streaming=True,
            max_new_tokens=500,
        )
        print("LLM set up successfully")
        return llm
    except Exception as e:
        print(f"Error setting up LLM: {e}")
        print("\nTroubleshooting steps:")
        print("1. Check if model files are properly downloaded")
        print("2. Verify the model path is correct")
        print("3. Ensure you have the right version of ExLlamaV2")
        print("4. Make sure you have enough GPU memory")
        raise

# 4. RAG Chain
def run_rag_chain(pdf_path, question, model_name="TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"):
    """Run a RAG chain to answer a question based on PDF content."""
    print(f"Processing question: '{question}'")

    # Process PDF
    chunks = load_pdf(pdf_path)
    vector_store = create_vector_store(chunks)

    # Retrieve relevant documents
    print("Retrieving relevant document chunks...")
    relevant_docs = vector_store.similarity_search(question, k=3)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    print(f"Retrieved {len(relevant_docs)} relevant chunks")

    # Setup LLM
    llm = setup_llm(model_name)

    # Create prompt template for RAG
    template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context:
    {context}

    Question: {question}

    Answer: Let's think step by step."""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    # Create and run chain
    print("Running LLM chain...")
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    output = llm_chain.invoke({"context": context, "question": question})

    return output

# Example usage
if __name__ == "__main__":
    pdf_path = "/content/The_Little_Prince_Antoine_de_Saint_Exupery.pdf"  # Replace with your PDF file path
    question = "What are the key points in this document?"  # Replace with your question

    try:
        result = run_rag_chain(pdf_path, question)
        print("\n\nFinal answer:")
        print(result['text'])
    except Exception as e:
        print(f"Error running RAG chain: {e}")

Processing question: 'What are the key points in this document?'
Loading PDF from /content/The_Little_Prince_Antoine_de_Saint_Exupery.pdf...
Loaded 54 pages from PDF
Split into 111 chunks
Creating embeddings...
Using device: cuda


<ipython-input-1-63236b36ae80>:78: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Creating vector store...
Vector store created successfully
Retrieving relevant document chunks...
Retrieved 3 relevant chunks
Setting up LLM...
Model already exists at /content/models/Mistral-7B-Instruct-v0.2-GPTQ
{'token_repetition_penalty': 1.05, 'token_repetition_range': -1, 'token_repetition_decay': 0, 'token_frequency_penalty': 0.0, 'token_presence_penalty': 0.0, 'temperature': 0.85, 'smoothing_factor': 0.0, 'min_temp': 0, 'max_temp': 0.0, 'temp_exponent': 1.0, 'top_k': 50, 'top_p': 0.8, 'top_a': 0.0, 'min_p': 0, 'tfs': 0, 'typical': 0, 'skew': 0, 'temperature_last': False, 'mirostat': False, 'mirostat_tau': 1.5, 'mirostat_eta': 0.1, 'mirostat_mu': None, 'token_bias': None, 'cfg_scale': None, 'post_sampling_hooks': [], 'dry_allowed_length': 2, 'dry_base': 1.75, 'dry_multiplier': 0.0, 'dry_sequence_breakers': None, 'dry_range': 0, 'dry_max_ngram': 20, 'ngram_trie': None, 'ngram_index': 0, 'ngram_history': deque([]), 'xtc_probability': 0.0, 'xtc_threshold': 0.1, 'xtc_ignore_tokens':

<ipython-input-1-63236b36ae80>:160: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


 The document describes a conversation between the little prince and a geographer. The little prince had drawn a picture of baobab trees and wanted to show it to the geographer. The geographer was impressed but wanted evidence of the discovery before he could acknowledge it. The little prince explained that he came from a distant planet and had no evidence to bring. The geographer then changed the subject to flowers and their ephemeral nature. The little prince was distressed thinking about his flower back home and how he had left her alone. The document also mentions that the little prince had tried to draw other grandiose pictures but had not succeeded. The key points are that the little prince had made a discovery but couldn't provide evidence, the geographer required evidence for acknowledgement, and the little prince was concerned about his flower back home. 1. In which part of the document does the little prince describe his flower as ephemeral?
2. What does the geographer say ab